# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компании «Хоть потоп». Требуется разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

# Описание данных

<b>Признаки:</b> 
<ul>
<li>пол</li>
<li>возраст</li>
<li>зарплата</li>
<li>количество леном семьи</li>
</ul>
<br>
<b>Целевой признак:</b> 
<ul>
<li>количество страховых выплат клиенту за последние 5 лет</li>
</ul>    

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from numpy.linalg import inv
from numpy.linalg import det

## Загрузка данных

In [2]:
data = pd.read_csv('insurance.csv')

In [3]:
data.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
data.duplicated().sum()

153

In [7]:
data = data.drop_duplicates()
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                4847 non-null   int64  
 1   Возраст            4847 non-null   float64
 2   Зарплата           4847 non-null   float64
 3   Члены семьи        4847 non-null   int64  
 4   Страховые выплаты  4847 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 227.2 KB
None


В данных нет пропусков, но есть дубликаты (153). От них мы избавились. Количество мужчин и женщин примерно одинаковое, средний возраст 31 год, средняя зарплата 39916, в среднем 1 ребенок в семье. Целевой признак - количество страховых выплат клиенту - является количественным, поэтому будем решать задачу регрессии.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** качество линейной регрессии не изменится

**Обоснование:**

В данном задании требуется доумножить признаки на обратимую матрицу, то есть на матрицу P.
Тогда формула обучения будет выглядеть следующим образом (для наглядности заменим $w$ на $w_1$)
$$
w_1 = ((XP)^T (XP))^{-1} (XP)^T y 
$$
$$
w_1 = (P^T(X^TX)P)^{-1}P^TX^Ty 
$$
$$
w_1 = P^{-1}(X^TX)^{-1}P^{T^{-1}}P^TX^Ty 
$$
$$
w_1 = P^{-1}(X^TX)^{-1}EX^Ty 
$$
$$
w_1 = P^{-1}(X^TX)^{-1}X^Ty 
$$
$$
w_1 = P^{-1}w
$$

Подставим признаки, умноженные на обратимую матрицу и новый вектор весов линейной легрессии в формулу для расчета MSE и получим следующую задачу обучения
$$
w = \arg\min_w MSE(XPP^{-1}w, y) = \arg\min_w MSE(XEw, y) = \arg\min_w MSE(Xw, y)
$$

Следовательно, качество линейной регрессии не изменится

## Алгоритм преобразования

In [8]:
features = data.drop('Страховые выплаты', axis = 1)
target = data['Страховые выплаты']

features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)

**Алгоритм**

Сначала требуется узнать размер features

In [9]:
print(features.shape)

(4847, 4)


В ходе преобразования данных следует умножить признаки на обратимую матрицу. Для этого создадим матрицу размером 4x4, так как для умножения матриц количество столбцов в первой матрице(в матрице признаков) должно совпадать с количеством строк во второй матрице (в обратимой матрице), а так как только квадратные матрицы являются обратимыми, то размер матрицы будет именно 4x4. Матрица будет заполняться случайными числами от 1 до 10000000 для лучшей защиты данных. Проверим обратимость матрицы, используя то свойство, что для того, чтобы у квадратной матрицы A была обратная матрица необходимо и достаточно чтобы определитель |A| был не равен нулю. Для этого воспользуемся функцией det() из пакета linalg. Далее построим модель для признаком умноженных на обратимую матрицу и предскажем значения признака. Посчитав метрику r2_score получим точно такую же величину, как и для признаков без умножения.

**Обоснование**

В предыдущем пункте было обосновано то, что при умножении признаков на обратиму матрицу результирующая метрика не изменит своего значения

## Проверка алгоритма

<b>Качество линейной регрессии с признаками без умножения</b>

In [10]:
class LinearRegression:
    def fit(self, features_train, target_train):
        X = np.concatenate((np.ones((features_train.shape[0], 1)), features_train), axis=1)
        y = target_train
        w = inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, features_valid):
        return features_valid.dot(self.w) + self.w0

In [11]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
print(r2_score(target_valid, predictions))

0.4230772749214826


<b>Качество линейной регрессии с признаками умноженными на обратную матрицу</b>

In [12]:
a = np.random.randint(1, 10000000, size=(4,4))
while det(a) == 0:
    a = np.random.randint(1, 10000000, size=(4,4))
features_encrypted = features.values.dot(a)
target_encrypted = data['Страховые выплаты']

features_train_encrypted, features_valid_encrypted, target_train_encrypted, target_valid_encrypted = train_test_split(features_encrypted, target, test_size=0.25, random_state=12345)

In [13]:
model = LinearRegression()
model.fit(features_train_encrypted, target_train_encrypted)
predictions = model.predict(features_valid_encrypted)
print(r2_score(target_valid_encrypted, predictions))

0.42307726697809034


## Вывод

В ходе работы было установлено то, что при умножении признаков на обратимую матрицу качество линейной регрессии не изменится. И для просто признаков и для признаков умноженных на обратимую матрицу получили метрику r2_score равную 0.423077.  В случае домножения матрицы признаков на случайную обратимую матрицу можно обеспечить защиту данных, не особо потеряв в качестве модели. (Незначительное отличие вызвано особенностью преобразования матриц и чисел с плавающей точкой)